In [3]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import requests
from bs4 import BeautifulSoup
import json
import os
from glob import *

In [21]:
path = os.getcwd()
path = path +'\\xG_Bundesliga\\Raw_Player_Stats\\'

dir = path
dateiname = []

for file in glob(dir + "/*.xlsx"):
   dateiname.append(file.split('.xlsx')[0].rpartition('_')[-1])
namen = np.array(dateiname, dtype=int)
int_low = np.max(namen)

#for i in range(int_low+1,int_low+9,1):
for i in range(17516,17822,1):

    [strings, strings2] = url_scrap(i)
    data = create_json(strings)
    [df, season, team, match_id] = create_df(data)
    create_excel(df, season, team, match_id)


[{'id': '425533', 'minute': '11', 'result': 'BlockedShot', 'X': '0.874000015258789', 'Y': '0.5429999923706055', 'xG': '0.020128345116972923', 'player': 'Robert Lewandowski', 'h_a': 'a', 'player_id': '227', 'situation': 'OpenPlay', 'season': '2021', 'shotType': 'Head', 'match_id': '17516', 'h_team': 'Borussia M.Gladbach', 'a_team': 'Bayern Munich', 'h_goals': '1', 'a_goals': '1', 'date': '2021-08-13 18:30:00', 'player_assisted': 'Josip Stanisic', 'lastAction': 'Aerial'}, {'id': '425534', 'minute': '13', 'result': 'SavedShot', 'X': '0.85', 'Y': '0.44299999237060544', 'xG': '0.05120028927922249', 'player': 'Robert Lewandowski', 'h_a': 'a', 'player_id': '227', 'situation': 'OpenPlay', 'season': '2021', 'shotType': 'RightFoot', 'match_id': '17516', 'h_team': 'Borussia M.Gladbach', 'a_team': 'Bayern Munich', 'h_goals': '1', 'a_goals': '1', 'date': '2021-08-13 18:30:00', 'player_assisted': 'Leon Goretzka', 'lastAction': 'HeadPass'}, {'id': '425536', 'minute': '25', 'result': 'SavedShot', 'X':

In [5]:
def url_scrap(id):
    
    base_url = 'https://understat.com/match/'
    match = str(id)
    url = base_url + match
    res = requests.get(url)
    soup = BeautifulSoup(res.content, 'lxml')
    scripts = soup.find_all('script')
    strings = scripts[1].string
    strings2 = scripts[2].string
 
    return strings, strings2

In [18]:
def create_json(strings):

    
    ind_start = strings.index("('")+2
    ind_end = strings.index("')")

    json_data = strings[ind_start:ind_end]
    json_data = json_data.encode('utf8').decode('unicode_escape')

    

    data = json.loads(json_data)
    print(data['a'])
    return data

In [7]:
def create_df(data):
    match_id = []
    player_assisted = []
    lastAction = []
    min = []
    result = []
    situation = []
    season = []
    date = []
    x = []
    y = []
    xG = []
    team = []
    player = []
    data_away = data['a']
    data_home = data['h']

    for index in range(len(data_home)):
        for key in data_home[index]:
            if key == 'X':
                x.append(data_home[index][key])
            if key == 'Y':
                y.append(data_home[index][key])
            if key == 'xG':
                xG.append(data_home[index][key])
            if key == 'h_team':
                team.append(data_home[index][key])
            if key == 'player':
                player.append(data_home[index][key])
            if key == 'minute':
                min.append(data_home[index][key])
            if key == 'result':
                result.append(data_home[index][key])
            if key == 'situation':
                situation.append(data_home[index][key])
            if key == 'season':
                season.append(data_home[index][key])
            if key == 'date':
                date.append(data_home[index][key])
            if key == 'match_id':
                match_id.append(data_home[index][key])
            if key == 'player_assisted':
                player_assisted.append(data_home[index][key])
            if key == 'lastAction':
                lastAction.append(data_home[index][key])

    for index in range(len(data_away)):
        for key in data_away[index]:
            if key == 'X':
                x.append(data_away[index][key])
            if key == 'Y':
                y.append(data_away[index][key])
            if key == 'xG':
                xG.append(data_away[index][key])
            if key == 'a_team':
                team.append(data_away[index][key])
            if key == 'player':
                player.append(data_away[index][key])  
            if key == 'minute':
                min.append(data_away[index][key])
            if key == 'result':
                result.append(data_away[index][key])
            if key == 'situation':
                situation.append(data_away[index][key])
            if key == 'season':
                season.append(data_away[index][key])
            if key == 'date':
                date.append(data_away[index][key])
            if key == 'match_id':
                match_id.append(data_away[index][key])
            if key == 'player_assisted':
                player_assisted.append(data_away[index][key])
            if key == 'lastAction':
                lastAction.append(data_away[index][key])

    
    col_name = ['min','x', 'y', 'xG', 'team', 'player', 'result', 'situation', 'season', 'date', 'match_id', 'player_assisted', 'lastAction']
    df = pd.DataFrame([min,x,y,xG,team,player,result,situation,season,date, match_id, player_assisted, lastAction], index=col_name)
    df = df.T
    
    return df, season, team, match_id         

In [8]:
def create_excel(df, season, team, match_id):
    team = np.array(team)
    team = np.unique(team)
    root = 'xG_Bundesliga\\Raw_Player_Stats'
    
    final_path = os.path.join(root, season[0]+'_'+team[0].replace(" ", "-")+'_'+team[1].replace(" ", "-")+'_'+match_id[0]+'.xlsx')
    df.to_excel(final_path)